# import, 함수

In [47]:
import pyautogui as au
import pyperclip
import time
import pandas as pd
import re
import pickle
from os.path import join
import os
import numpy as np
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
import random
# 정렬순서 : 확정일자(내림), 법원(올림), 사건번호(내림)

# pause
def lwt() : time.sleep(2.5)
def mwt() : time.sleep(1)
def swt() : time.sleep(random.uniform(0.1,0.4))
# 법원선택
def gjlc() :
    au.click(x=2893, y=458)
    au.moveTo(x=2998, y=498)
    au.mouseDown() # 좌클릭
    au.moveTo(x=2999, y=690)
    au.mouseUp()
    au.click(x=2894, y=714)

def bslc() :
    au.click(x=2893, y=458)
    au.moveTo(x=2998, y=498)
    au.mouseDown() # 좌클릭
    au.moveTo(x=2999, y=662)
    au.mouseUp()
    au.click(x=2900, y=624)

def con_scourt() :
    service = Service(executable_path="./파일/chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    driver.get("https://ecfs.scourt.go.kr/ecf/index.jsp")
    wait = WebDriverWait(driver, 30) 
    wait.until(EC.element_to_be_clickable((By.ID, 'userId1')))
    mwt()
    return driver, wait

def certify() :
    elem = wait.until(EC.element_to_be_clickable((By.ID, "xwup_certselect_tek_input1")))
    mwt()
    elem.send_keys("help8700765!")
    swt()
    driver.find_element(By.ID, 'xwup_OkButton').click() # 인증서 확인
    mwt()

def login() : 
    # 로그인(id) # 셀레니움 안 돼서.. 
    position = driver.get_window_position()
    x = position.get('x')
    y = position.get('y')
    au.click(x+100,y+305)
    time.sleep(0.2)
    au.write("help")
    au.press("num2")
    au.press("num2")
    au.write("55")
    driver.find_element(By.ID, "userId1").send_keys(Keys.ENTER)
    # 로그인(인증서)
    certify()
    lwt()

def certificates(apply_or_print : int, docu_kind : int) :
    """apply_or_print : 신청 : 1, 발급 : 2 \n
    docu_kind : 확정증명:0 | 송달증명:1 | 집행문:3"""
    
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gnb"]/ul/li[4]/a'))).send_keys(Keys.ENTER) # 제증명
    driver.find_element(By.XPATH, '//*[@id="tmenu04"]/div/ul/li['+str(apply_or_print)+']/a').send_keys(Keys.ENTER) # 제증명신청
    swt()
    Select(driver.find_element(By.ID, 'cbxDocCd')).select_by_index(docu_kind) # 확정증명 선택
    swt()


def select_court_event(court, event) :# driver
    # 법원
    correct_court = re.sub("지법", "지방법원", court)
    Select(driver.find_element(By.ID, 'cbxBubCd')).select_by_visible_text(correct_court) # 집행문 선택
    swt()
    # 사건번호 선택
    event = re.search("(\d{4,4})([가-힣]+)(\d+)", event)
    year = event[1]
    sign = event[2]
    sn = event[3]
    Select(driver.find_element(By.ID, 'cbxSaYear')).select_by_value(year) # 연도 선택
    Select(driver.find_element(By.ID, 'cbxSaGbncd')).select_by_visible_text(sign)  # 사건구분 선택
    driver.find_element(By.ID, 'txtSaSerial').send_keys(sn) # 일련번호 선택
    swt()
    driver.find_element(By.ID, 'btnConfirm').click() # confirm
    mwt()
    return

def select_parties_new_win() : ##############
    # 발급당사자 버튼
    wait.until(EC.element_to_be_clickable((By.ID, 'btnSelBalDs'))).click() # 발급당사자선택 버튼
    mwt()

    # 새창 전환
    driver.switch_to.window(driver.window_handles[1]) # 창 전환
    # 팝업창 컨트롤(발급당사자)
    driver.find_element(By.ID, 'allCheck').click() # 당사자 선택
    driver.find_element(By.ID, 'btnReg').click() # 등록
    swt()
    driver.find_element(By.ID, 'selectedAllCheck').click()  # 선택된 당사자 선택
    driver.find_element(By.ID, 'btnSetDs').click() # 확인
    swt()
    # 창 되돌리기
    driver.switch_to.window(driver.window_handles[0]) # 창 전환
    return 

def check_document() : #############
    time.sleep(7) # pdf문서 대기
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="pdfReader"]/iframe')))
    
    # 문서 내용 확인 및 스크롤 (안 되네... )
    # driver.switch_to.frame(driver.find_elements(By.XPATH, '//*[@id="pdfReader"]/iframe')) # 문서viewer iframe 전환
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # 문서 끝까지 스크롤 
    # driver.find_element(By.XPATH, '//*[@id="body"]/sd-root/sd-main/sd-toolbar/section[3]/section[2]/div/pu-button/div/pu-icon').click()
    
    # 문서 이상없음 체크 및 확인버튼
    driver.find_element(By.ID, 'guidcheck').click() # 문서이상없음
    driver.find_element(By.ID, 'btnNextPage').click() # 확인
    mwt()

def send_confirm() :
    # 제출 및 확인창
    wait.until(EC.element_to_be_clickable((By.ID, 'btnSendJchul'))).click() # 제출 버튼
    swt()
    Alert(driver).accept() # 확인alert
    mwt()


#############################
driver, wait = con_scourt()
login() # 로그인 : 수동으로 하자. 인증서 되는 거 확인

# 집행문

In [55]:
df = pd.read_excel(r"D:\0.전산\관리팀업무\전자소송\221215\집행문신청_221209~221215.xlsx").fillna("")
df
# NaN 값을 조건식으로 쓰고 싶으면 str( ) == "nan" 으로 처리

,번호,법원,사건번호,재판부,확정일자,원고,피고,집행문,송달,확정,비고,집행비용,가상계좌번호
0,1,부산지법,2022가소605974,민사제28단독,2022.12.10,주식회사 솔림헬프자산관리대부,정순학,신청,,,,,56216072432650.0
1,2,부산지법,2022가소605394,민사제28단독,2022.12.10,주식회사 솔림헬프자산관리대부,주계송,신청,,,,,56216072443874.0
2,3,부산지법,2022가소598168,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,차남권,신청,,,,,56216072444699.0
3,4,부산지법,2022가소598182,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,이동건,신청,,,,,56216072446158.0
4,5,부산지법,2022가소598199,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,정남희,신청,,,,,56216072446158.0
5,6,부산지법,2022가소598205,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,김영훈,신청,,,,,56216072466407.0
6,7,부산지법,2022가소598212,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,정민호,신청,,,,,56216072467842.0
7,8,부산지법,2022가소598229,민사제27단독,2022.12.10,주식회사 솔림헬프자산관리대부,천태기,,,,,,
8,9,부산지법,2022가소598243,민사제27단독,2022.12.10,주식회사 솔림헬프자산관리대부,조종곤,,,,,,
9,10,부산지법,2022가소598281,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,유예슬,,,,,,


In [44]:
# 대법원 조회
driver, wait = con_scourt()
bank = "신한은행"
for i, v in df.iterrows() :
    if v.집행문 == "" :
        # 제증명 신청(1), 집행문(4)
        certificates(1, 3)
        # 법원 및 사건번호 및 확인
        select_court_event(v.법원, v.사건번호)
        # 발급당사자
        select_parties_new_win() # 발급당사자 새창

        # 동의 및 작성완료
        driver.find_element(By.ID, 'chkAgrElcSo').click() # 동의 체크박스
        driver.find_element(By.ID, 'btnComplete').click() # 작성완료
        mwt()
        Alert(driver).accept() # 확인창
        
        # 문서이상없음 및 확인
        check_document() 

        # 은행선택 및 납부
        Select(driver.find_element(By.ID, 'vtBankCd')).select_by_visible_text(bank) # 은행
        driver.find_element(By.ID, 'btnElcPay').click()
        swt()
        # 확인창 확인 클릭
        Alert(driver).accept()
        mwt()
        
        # 제출 및 confirm 확인
        send_confirm()
            
        # 인증서
        certify()
        lwt()

        # 계좌정보 담기 및 완료버튼
        account = driver.find_element(By.ID, 'acno').text # 계좌정보 담기
        wait.until(EC.element_to_be_clickable((By.ID, 'btnSendList'))).click() # 완료버튼
        swt()
        #driver.find_element(By.XPATH, '//*[@id="cbxDocCd"]/option[4]"]').send_keys(Keys.ENTER) # 확인alert 이거 없나?

        # selection = input()
        # if selection == "0" : pass
        # else : break

        # df 수정 : 계좌번호 및 집행문 상태
        v["가상계좌번호"] = account
        v["집행문"] = "신청"

df
        

,번호,법원,사건번호,재판부,확정일자,원고,피고,집행문,송달,확정,비고,집행비용,가상계좌번호
0,1,부산지법,2022가소605974,민사제28단독,2022.12.10,주식회사 솔림헬프자산관리대부,정순학,신청,nan,nan,nan,nan,56216072432650.0
1,2,부산지법,2022가소605394,민사제28단독,2022.12.10,주식회사 솔림헬프자산관리대부,주계송,신청,nan,nan,nan,nan,56216072443874.0
2,3,부산지법,2022가소598168,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,차남권,신청,nan,nan,nan,nan,56216072444699.0
3,4,부산지법,2022가소598182,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,이동건,신청,nan,nan,nan,nan,56216072446158.0
4,5,부산지법,2022가소598199,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,정남희,신청,nan,nan,nan,nan,56216072446158.0
5,6,부산지법,2022가소598205,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,김영훈,신청,nan,nan,nan,nan,56216072466407.0
6,7,부산지법,2022가소598212,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,정민호,신청,nan,nan,nan,nan,56216072467842.0
7,8,부산지법,2022가소598229,민사제27단독,2022.12.10,주식회사 솔림헬프자산관리대부,천태기,신청,nan,nan,nan,nan,56216072468258
8,9,부산지법,2022가소598243,민사제27단독,2022.12.10,주식회사 솔림헬프자산관리대부,조종곤,신청,nan,nan,nan,nan,56216072468347
9,10,부산지법,2022가소598281,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,유예슬,신청,nan,nan,nan,nan,56216072468420


In [61]:
df.to_excel(r"D:\0.전산\관리팀업무\전자소송\221215\집행문신청_221209~221215_result.xlsx", index=False)

### pygui

In [ ]:
        # au.click(x=2916, y=223)
        # time.sleep(0.1)
        # au.click(x=2929, y=257)
        # time.sleep(2.5)
        # # 제증명 종류 : 집행문 선택
        # au.click(x=2923, y=407)
        # time.sleep(0.1)
        # au.click(x=2887, y=479)
        # # 법원 : 부산지법 선택
        # au.click(x=2893, y=458)
        # au.moveTo(x=2998, y=498)
        # au.mouseDown() # 좌클릭
        # au.moveTo(x=2999, y=662)
        # au.mouseUp()
        # au.click(x=2900, y=624)
        # # 사건구분자
        # if v.사건구분 == "가소" :
        #     au.click(x=2916, y=480)
        #     au.click(x=2912, y=532)
        # # 일련번호 입력
        # au.click(x=2973, y=485)
        # au.write(str(v.일련번호))
        # au.press("enter")
        # selection = input()
        # if selection == "0" :
        #     pass
        # else :
        #     break

In [65]:
x = pd.DataFrame([1,2,3], columns=["숫자"])

for k, v in x.iterrows() :
    v.숫자 = v + 3
    break
x

,숫자
0,4
1,2
2,3


# 확정증명원/송달증명원

In [54]:
df = pd.read_excel(r"D:\0.전산\관리팀업무\전자소송\221215\집행문신청_221209~221215.xlsx").fillna("")
df = df[16:] ######################
df

,번호,법원,사건번호,재판부,확정일자,원고,피고,집행문,확정,송달,비고,집행비용,가상계좌번호
16,17,광주지법,2018가소546633,,,주식회사 솔림헬프자산관리대부,김수남 외 1명,발급,신청,,재도,,
17,18,광주지법,2018가소548363,,,주식회사 솔림헬프자산관리대부,박순자,발급,신청,,재도,,
18,19,광주지법,2021가소521887,,,주식회사 솔림헬프자산관리대부,백재동,발급,신청,,재도,,


In [55]:
def fix_delivery(which:int, v) :
    "which : 확정 - 0, 송달 - 1 \t v : df.iterrows()[1]"
    certificates(1, which) # 제증명신청, 제증명 종류
    select_court_event(v.법원, v.사건번호) # 법원 사건번호 확인
    wait.until(EC.element_to_be_clickable((By.ID, "sinGbnCd2"))).click() # 당사자별
    swt()
    select_parties_new_win() # 발급당사자선택 ~ 새창 컨트롤
    driver.find_element(By.ID, "btnComplete").click() # 작성완료
    mwt()
    check_document() # 문서확인
    send_confirm() # 제출 및 comfirm
    certify() # 인증서 
    lwt() # 제출 후 인증은 lwt()추가!
    wait.until(EC.element_to_be_clickable((By.ID, 'btnSendList'))).click() # 완료버튼
    swt()
    # df수정
    if which == 0 : v.확정 = "신청"
    elif which == 1 : v.송달 = "신청"

In [56]:
for i, v in df.iterrows() :
    if v.확정 == "" :
        fix_delivery(0, v)
    if v.송달 == "" :
        fix_delivery(1, v)

### 확정증명원(pygui)

In [ ]:
##### pygui

lwt = 2.5
mwt = 1
swt = 0.1

# 확정증명원
for i, v in dfdf.iterrows() :
    event = re.search("[가-힣]+", v.사건번호).group()
    sn = re.search("[가-힣]+(\d+)", v.사건번호).group(1)
    # 제증명 신청
    au.click(x=2916, y=223)
    time.sleep(swt)
    au.click(x=2929, y=257)
    time.sleep(lwt)
    # 제증명 종류 : 확정증명
    # 법원 : 부산지법 선택
    if v.법원 == "광주지법" : gjlc()
    elif v.법원 == "부산지법" : bslc()
    # 사건구분자
    if event == "가소" :
        au.click(x=2916, y=480)
        au.click(x=2912, y=532)

    # 일련번호 입력
    au.click(x=2973, y=485)
    au.write(sn)
    au.press("enter")
    time.sleep(lwt)

    # 선택 
    au.click(x=3051, y=759)
    time.sleep(mwt)
    # for win in au.getWindowsWithTitle('사건공증정보 선택') :
    #     if win.isActive == False : win.activate()
    # au.click(win.left+36, win.top+177)
    au.click(x=1959, y=180)
    au.press("\t")
    au.press("enter")
    time.sleep(swt)
    au.click(x=3271, y=945)
    #문서확인
    time.sleep(lwt)
    time.sleep(lwt)
    au.click(x=2392, y=581) # 빈화면 클릭
    au.press('pagedown')
    au.press('pagedown')
    au.click(x=2995, y=771)
    time.sleep(swt)
    au.click(x=3279, y=882)
    # 제출
    time.sleep(mwt)
    au.click(x=3271, y=703)
    time.sleep(swt)
    au.press('enter') # 알람창 확인
    time.sleep(mwt)
    time.sleep(mwt)
    
    # 인증서
    au.click(x=2845, y=773)
    
    # 완료버튼
    au.sleep(lwt)

    au.click(x=732, y=50) # 집행문.ipynb 클릭(탐색기 있고 3번째 탭)
    selection = input()

    if selection == "0" :
        pass
    else :
        break

### 송달증명원(pygui)

In [ ]:
# 송달증명원
right_complete_url = "https://ecfs.scourt.go.kr/ecf/ecf300/ECF320_F.jsp"
for i, v in dfdf.iterrows() :
    
    event = re.search("[가-힣]+", v.사건번호).group()
    sn = re.search("[가-힣]+(\d+)", v.사건번호).group(1)
    # 제증명 신청
    au.click(x=2916, y=223)
    swt()
    au.click(x=2929, y=257)
    lwt()
    # 제증명 종류 : 확정증명
    au.click(x=2923, y=407)
    swt()
    au.click(x=2908, y=440)
    # 법원 : 부산지법 선택
    if v.법원 == "광주지법" : gjlc()
    elif v.법원 == "부산지법" : bslc()
    # 사건구분자
    if event == "가소" :
        au.click(x=2916, y=480)
        au.click(x=2912, y=532)

    # 일련번호 입력
    au.click(x=2973, y=485)
    au.write(sn)
    au.press("enter")
    lwt()

    # 발급당사자선택 
    au.click(x=3239, y=785)
    mwt()

    # 팝업창
    au.click(x=1953, y=230) # 전체선택
    au.click(x=2400, y=333) # 등록
    mwt()
    au.click(x=2205, y=343) # 빈화면
    au.press("pagedown")
    swt()
    au.click(x=1953, y=296) # 전체
    au.click(x=2340, y=435) # 확인
    mwt()

    # 작성완료
    au.click(x=2444, y=826)
    au.press("pagedown")
    swt()
    au.click(x=3264, y=743)
   
    #문서확인
    lwt()
    lwt()
    au.click(x=2392, y=581) # 빈화면 클릭
    au.press('pagedown')
    au.press('pagedown')
    au.click(x=2995, y=771) # 이상없음
    swt()
    au.click(x=3279, y=882) # 확인
    # 제출
    mwt()
    au.click(x=3271, y=703)
    swt()
    au.press('enter') # 알람창 확인
    mwt()
    mwt()
    
    # 인증서
    au.click(x=2845, y=773)
    
    # 완료버튼
    lwt()
    lwt()
    lwt()

    # url을 통해 계속하기
    au.click(x=2431, y=475)
    au.hotkey("alt", "d")
    au.hotkey("ctrl", "c")
    swt()
    current_url = pyperclip.paste()
    
    if current_url !=  right_complete_url : 
        break
    else :
        print(f"{len(dfdf.index)}개 중 {i+1}번째, 사건번호 : {v.사건번호}")


7개 중 33번째, 사건번호 : 2022가소582286
7개 중 34번째, 사건번호 : 2022가단331446
7개 중 35번째, 사건번호 : 2022가단331453
7개 중 36번째, 사건번호 : 2022가단331460
7개 중 37번째, 사건번호 : 2022가단325946
7개 중 38번째, 사건번호 : 2022가단324738
7개 중 39번째, 사건번호 : 2022가소541063


# 출력

In [11]:
# 제증명 발급
au.click(x=2916, y=223)
swt()
au.click(x=2925, y=282)
lwt()
lwt()
mwt()
for i in range(51,90) : # (page번호(1~10)와 post번호(0~9)+1)
    
    page_no = (i-10)//10 # 10번째의 몫이 1이 되어버리는 문제
    post_no = i%10 

    # 게시판 이동
    if page_no > 0 :
        au.moveTo(x=2890, y=856) # 1번페이지 다 발급완료한 경우의 1번페이지 위치
        au.move(16.444*page_no, 0) 
        au.click()
        lwt()
        mwt()

    # 게시물에 맞는 발급버튼 누르기
    au.moveTo(x=3258, y=586)
    au.move(0, 26*(post_no))
    swt()
    au.click() # 발급
    mwt()
    au.press('enter')

    lwt()
    lwt()
    mwt()
    au.click(x=1143, y=826) # 인쇄
    lwt()
    au.press('enter')
                            
    lwt() # 로딩
    lwt()
    print(i)
    #selection = au.confirm(text=i, title='continue?', buttons=['OK', 'Cancel'])

    #if selection == 'Cancel' : break
    



51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


# 법조치 등록 관련

In [11]:
# cols = 채무자키, 매각사구분, 법조치구분, 관할법원, 사건번호, 청구금액법원, 관련사건번호, 매각사
path_xlsx = r'D:\0.전산\관리팀\전자소송\법조치 등록' 
path_pkl = path_xlsx
##########################################
file_name = "법조치조회새창_참조용_20221207_1309"
###########################################
df_law = pd.read_excel(join(path_xlsx, file_name + ".xlsx"))
df_law.to_pickle(join(path_pkl, file_name + ".pkl"))
# pkl 읽기
df_law = pd.read_pickle(join(path_pkl, file_name + ".pkl"))
df_law.head(1)

c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,채무자키,매각사구분,법조치키,법조치구분,법조치세부,관할법원,사건번호,비고,계좌키,접수일,...,법조치기간,관련사건법원,관련사건번호,매각사,타법조치키,원채권사,법조치메모,제소신청일,등록일자,등록유저
0,20407790,DNP-01,20678164,지급명령,지급명령 신청,서울중앙지방법원,2014차전240794,NaN,200919485,2014-10-24,...,NaN,NaN,NaN,디엔피에이엠씨대부㈜,25348.0,외환카드,NaN,NaN,2019-01-02,system


In [14]:
path = r"D:\scan\새 폴더"
p_e = re.compile("\d{4}[가-힣]{1,3}\d+")
df_result = pd.DataFrame(columns=["채무자키", "계좌키", "사건번호", "계좌키", "제소신청일"])
for f in os.listdir(path) :
    stem, ext = os.path.splitext(f)
    e_name = p_e.search(stem).group()
    # drop_duplicates(subset=["채무자키"], keep='first').
    keys = df_law[(df_law.관련사건번호== e_name)].loc[:,["채무자키", "계좌키", "사건번호", "계좌키", "제소신청일"]]
    df_result = pd.concat([df_result, keys])
    
df_result    

,채무자키,계좌키,사건번호,계좌키,제소신청일
77541,20430726,200951530,2022차전113757,200951530,2022-08-16
77780,20430830,200951636,2022차전112848,200951636,NaN
78054,20430953,200951763,2022차전114681,200951763,NaN
78106,20430975,200951785,2022차전112741,200951785,NaN
78451,20431140,200951954,2022차전114935,200951954,NaN
78513,20431169,200951983,2022차전117005,200951983,NaN
78746,20431292,200952110,2022차전116543,200952110,NaN
78797,20431318,200952136,2022차전115770,200952136,2022-09-13
79922,20431885,200952760,2022차전113758,200952760,2022-08-16
80222,20432050,200952974,2022차전102910,200952974,2022-04-14


In [17]:
df_result.to_excel('c:/Users/SL/Desktop/참조.xlsx')

In [16]:
p_e = re.compile("\d{4}[가-힣]{1,3}\d+")

for f in os.listdir(path) :
    key =f[:8]
    event = p_e.search(f).group()
    print(key, event, "\n")

20430726 2022가소582033 

20430830 2022가소581900 

20430953 2022가소582286 

20430975 2022가소585179 

20431140 2022가소592368 

20431169 2022가소587915 

20431292 2022가소592450 

20431318 2022가소592399 

20431885 2022가소582040 

20432050 2022가소541063 

20433086 2022가소582026 

20433576 2022가소592405 

20434160 2022가소587830 

20434239 2022가소592474 

20434856 2022가소582873 

20435270 2022가소585278 

20435299 2022가소584039 

20496518 2022가소592467 



# 셀레니움

In [8]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

def document_initialised(driver):
    return driver.execute_script("return initialised")



In [87]:
service = Service(executable_path="./파일/chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get("https://ecfs.scourt.go.kr/ecf/index.jsp")
driver.implicitly_wait(10) # 요소 정보를 로드하는 것과 실제 화면에 보이는 것은 차이가 있네.. time.sleep필요
#login(driver) # 로그인 : 수동으로 하자. 인증서 되는 거 확인

In [82]:
driver.quit()